Investigating the $L_{\boldsymbol y}$ matrix
==

In [ ]:
%matplotlib inline
import mushi
import histories
import utils
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from numpy.linalg import cond, svd
from matplotlib.colors import LogNorm

In [ ]:
# set this to e.g. your Downloads folder path if you want plots saved to pdfs
plot_dir = None

### define $\eta(t)$

In [ ]:
# time grid
t = np.concatenate(([0], np.logspace(0, 5, 1000), [np.inf]))
# η(t) values in each epoch
y = 1e4 * np.ones(len(t) - 1)
y[600:700] = 1e3
# histories object
eta = histories.eta(t[1:-1], y)

plt.figure(figsize=(3.5, 3.5))
eta.plot()
plt.show()

### Singular value spectrum of $L_{\mathbf y}$ with $n=200$

In [ ]:
n = 200
L = utils.C(n) @ utils.M(n, t, y)
U, σ, Vh = svd(L, full_matrices=False)
plt.figure(figsize=(3, 2))
plt.plot(np.arange(1, min(n, eta.m + 1)), σ, 'k.', markersize=3)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('singular value rank')
plt.ylabel('singular value')
plt.tight_layout()
if plot_dir:
    plt.savefig(f'{plot_dir}/L.svs.pdf')
plt.show()

### Top few right singular vectors

In [ ]:
df = pd.DataFrame(Vh.T, index=pd.Index(t[:-1], name='time'))
df = df.melt().set_index('variable')
df['singular value'] = σ[df.index]
df['$t$ (generations)'] = np.tile(t[:-1], df.index[-1] + 1)

# filter to top few singular values
df = df[(df.index % 1 == 0) & (df.index < 30)]

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, sharex='col', figsize=(4.4, 4))
plt.sca(axes[0])
eta.plot(lw=3, alpha=0.75)
plt.tick_params(axis='y', labelcolor='C0')
plt.xlabel('')
ax2 = axes[0].twinx()
plt.sca(ax2)
plt.plot(eta.change_points, mushi.kSFS(n=n).tmrca_cdf(eta), color='C1', lw=3, alpha=0.75)
plt.ylim([0, 1])
plt.ylabel('TMRCA CDF')
plt.tick_params(axis='y', labelcolor='C1')
plt.sca(axes[1])
ax = sns.lineplot(x='$t$ (generations)', y='value', hue='singular value', units='singular value',
                  data=df, estimator=None,
                  palette=sns.color_palette("RdBu", n_colors=df['singular value'].nunique()),
                  legend=False, alpha=.5, lw=1)
plt.xscale('log')
plt.ylim([-.2, .2])
plt.tight_layout()
if plot_dir:
    plt.savefig(f'{plot_dir}/L.pdf', transparent=True)
plt.show()